# Running the Gasperini 2019 pilot (small set using) using the CRISPR-fg PIPELINE

### current pipeline repository:
https://github.com/LucasSilvaFerreira/pipeline_perturbseq_like/tree/master

In [20]:
%cd /n/scratch3/users/l/lf114/gasperini_2019_pipeline_crispr


### Downloading Dataset
This is can take a few hours   (You can add more threads to speed up)  
adapt this script to your server case you need to send a job  

In [17]:
%%writefile download.sh
#!/bin/bash
#SBATCH -n 20                              # 10 core
#SBATCH -t 0-07:00                         # in D-HH:MM format
#SBATCH -p short                           # Run in short partition
#SBATCH --mem=80G
#SBATCH -o download.out
#SBATCH -e download.err
#SBATCH --mail-type=ALL                    # Type of email notification- BEGIN,END,FAIL,ALL
#SBATCH --mail-user=bioinfolucas@gmail.com   # Email to which notifications will be sent

NTHREADS=20; 
mkdir /n/scratch3/users/l/lf114/gasperini_2019_pipeline_crispr
cd /n/scratch3/users/l/lf114/gasperini_2019_pipeline_crispr
wget https://sra-pub-src-1.s3.amazonaws.com/SRR7967482/pilot_highmoi_screen.1_SI_GA_G1.bam.1;mv pilot_highmoi_screen.1_SI_GA_G1.bam.1 pilot_highmoi_screen.1_SI_GA_G1.bam
./bamtofastq_linux --nthreads="$NTHREADS" pilot_highmoi_screen.1_SI_GA_G1.bam bam_pilot_scrna_1
wget https://github.com/10XGenomics/bamtofastq/releases/download/v1.4.1/bamtofastq_linux; chmod +x bamtofastq_linux
wget https://sra-pub-src-1.s3.amazonaws.com/SRR7967488/pilot_highmoi_screen.1_CGTTACCG.grna.bam.1;mv pilot_highmoi_screen.1_CGTTACCG.grna.bam.1 pilot_highmoi_screen.1_CGTTACCG.grna.bam
./bamtofastq_linux --nthreads="$NTHREADS" pilot_highmoi_screen.1_CGTTACCG.grna.bam bam_pilot_guide_1


Overwriting download.sh


In [18]:
#sending as a job using jupyter (remove ! case in terminal)
!sbatch download.sh


Submitted batch job 19102404


#  Use this set of commands in the terminal
```bash
conda create -n perturbseq_like_pipeline_2023 python=3.8  
conda activate perturbseq_like_pipeline_2023  
conda install -c conda-forge mamba -y   
pip install nextflow  
pip uninstall GTFProcessing -y  
pip install git+https://github.com/LucasSilvaFerreira/GTFProcessing.git  
pip install gtfparse==1.3.0  
pip install git+https://github.com/LucasSilvaFerreira/Perturb_Loader.git  
pip install --quiet kb-python  
conda install -c conda-forge mamba -y  
mamba install -c bioconda nextflow -y  
mamba install -c bioconda kallisto -y  
mamba install -c anaconda openpyxl -y  
mamba install -c conda-forge r-base -y  
mamba install -c conda-forge r-gert -y  
mamba install -c conda-forge r-ragg -y  
mamba install -c conda-forge r-ggplot2 -Y  
mamba install -c conda-forge r-biocmanager -Y   
Rscript -e 'BiocManager::install("Rhdf5lib")'  
Rscript -e 'BiocManager::install("rhdf5")'  
Rscript -e 'BiocManager::install("ShortRead")'  
Rscript -e 'install.packages("doParallel", repos = "http://cran.us.r-project.org")'  
mamba conda install -c conda-forge r-devtools -y 
Rscript -e 'devtools::install_github("katsevich-lab/sceptre")'  
Rscript -e 'devtools::install_github("chris-mcginnis-ucsf/MULTI-seq")'  
pip install muon  
pip install scrublet  
pip install pybiomart  
mamba create -n pygenomictracks pygenometracks==3.8 -c bioconda -y  
```


### Note: This installation should be improved


In [16]:
#download white list
!wget https://github.com/10XGenomics/cellranger/raw/master/lib/python/cellranger/barcodes/737K-august-2016.txt

--2023-10-06 12:14:42--  https://github.com/10XGenomics/cellranger/raw/master/lib/python/cellranger/barcodes/737K-august-2016.txt
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/10XGenomics/cellranger/master/lib/python/cellranger/barcodes/737K-august-2016.txt [following]
--2023-10-06 12:14:42--  https://raw.githubusercontent.com/10XGenomics/cellranger/master/lib/python/cellranger/barcodes/737K-august-2016.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12533760 (12M) [text/plain]
Saving to: ‘737K-august-2016.txt’

100%[======================================>] 12,533,760  9.06MB/s   in 1.3s  

In [ ]:
# Capturing kallisto path bin
#use it in your config
!source activate perturbseq_like_pipeline_2023; which kallisto

In [18]:
!git clone https://github.com/LucasSilvaFerreira/pipeline_perturbseq_like.git

Cloning into 'pipeline_perturbseq_like'...
remote: Enumerating objects: 315, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 315 (delta 74), reused 47 (delta 20), pack-reused 183
Receiving objects: 100% (315/315), 1.25 MiB | 5.81 MiB/s, done.
Resolving deltas: 100% (173/173), done.


In [19]:
!ls pipeline_perturbseq_like

bin						    perturb.config
df_from_gasperini_tss.xlsx			    README.md
feature_example.xlsx				    run_jamboree_v2.ipynb
image						    taks_jamboree
main.nf						    testing.txt
merged_hbg_genes_perturb_cis_hbb_multi_high.config  Untitled1.ipynb
perturb_cis_hbb_low.config


In [ ]:
%%write gasperini_sample.config

params.GTF_GZ_LINK = 'http://ftp.ensembl.org/pub/release-106/gtf/homo_sapiens/Homo_sapiens.GRCh38.106.gtf.gz'
params.TRANSCRIPTOME_REFERENCE = "human"
params.KALLISTO_BIN = 'ADD YOUR KALLISTO BIN' //PATHWAY KALLISTO INSTALL check your install path (previous command)
params.GENOME = 'https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz'


params.GUIDE_FEATURES = 'pipeline_perturbseq_like/df_from_gasperini_tss.xlsx'



params.CHEMISTRY = "10XV3"
params.THREADS = 15
params.DISTANCE_NEIGHBORS = 1000000
params.IN_TRANS = "FALSE"
params.FASTQ_FILES_TRANSCRIPTS = ['bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L001_R1_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L001_R2_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L001_R1_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L001_R2_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L001_R1_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L001_R2_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HWVT7BGX3/bamtofastq_S1_L001_R1_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HWVT7BGX3/bamtofastq_S1_L001_R2_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L002_R1_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L002_R2_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L002_R1_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L002_R2_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L002_R1_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L002_R2_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HWVT7BGX3/bamtofastq_S1_L002_R1_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HWVT7BGX3/bamtofastq_S1_L002_R2_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L003_R1_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L003_R2_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L003_R1_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L003_R2_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L003_R1_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L003_R2_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HWVT7BGX3/bamtofastq_S1_L003_R1_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HWVT7BGX3/bamtofastq_S1_L003_R2_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L004_R1_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L004_R2_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L004_R1_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L004_R2_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L004_R1_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L004_R2_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HWVT7BGX3/bamtofastq_S1_L004_R1_001.fastq.gz  bam_pilot_scrna_1/K1000_CRISPRi_cells_r1_SI-GA-G1_MissingLibrary_1_HWVT7BGX3/bamtofastq_S1_L004_R2_001.fastq.gz'] 
params.FASTQ_NAMES_TRANSCRIPTS = ['S1_L1'] 



params.CUSTOM_REFERENCE = false
params.CUSTOM_REFERENCE_IDX = ''
params.CUSTOM_REFERENCE_T2T = ''
params.CUSTOM_GTF_PATH = ''


params.FASTQ_FILES_GUIDES = ['bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L001_R1_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L001_R2_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L001_R1_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L001_R2_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L001_R1_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L001_R2_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L002_R1_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L002_R2_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L002_R1_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L002_R2_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L002_R1_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L002_R2_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L003_R1_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L003_R2_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L003_R1_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L003_R2_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L003_R1_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L003_R2_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L004_R1_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMKLBGX3/bamtofastq_S1_L004_R2_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L004_R1_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HMMMLBGX3/bamtofastq_S1_L004_R2_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L004_R1_001.fastq.gz  bam_pilot_guide_1/K1000_CRISPRi_gRNA_r1_CGTTACCG_MissingLibrary_1_HW5MKBGX3/bamtofastq_S1_L004_R2_001.fastq.gz'] 
params.FASTQ_NAMES_GUIDES = ['S1_L1'] 

params.CREATE_REF = false
params.ADDGENENAMES = ''
params.DIRECTION = 'both'

params.WHITELIST= '737K-august-2016.txt'

params.EXPECTED_CELL_NUMBER = 10000
params.MITO_SPECIE = 'hsapiens'
params.MITO_EXPECTED_PERCENTAGE = 0.2
params.PERCENTAGE_OF_CELLS_INCLUDING_TRANSCRIPTS=0.01
params.TRANSCRIPTS_UMI_TRHESHOLD = 2000
params.GUIDE_UMI_LIMIT = 3

params.MERGE = false


//MULTISEQ

params.RUN_MULTISEQ = false
params.R1_MULTI = ''
params.R2_MULTI = ''
params.BARCODES_MULTIBAR_LIST_MULTI = ''
params.BAR_MULTI= [1,16]
params.UMI_MULTI= [17,28]
params.R2_MULTI_TAG = [1,8]

/home/lf114/miniconda3/envs/perturbseq_pipeline/envs/perturbseq_like_pipeline_2023/bin/kallisto


In [ ]:

execute_path = '/n/scratch3/users/l/lf114/gasperini_2019_pipeline_crispr' # your path
nf_path = 'pipeline_perturbseq_like/main.nf'    #'https://github.com/LucasSilvaFerreira/pipeline_perturbseq_like'
dir_out = 'gasperini_test_01'
report_path = f'{dir_out}'
resume = '-resume' # -resume or empty
param_file_path = 'gasperini_sample.config'
visualization_timeline = f'{dir_out}.html'
#case WANT TO have an amazing dashboard
tower_toker = 'aaa' # create your own account in next tower and capture your token https://cloud.tower.nf
next_tower =   f'export TOWER_ACCESS_TOKEN={tower_toker}' # add your next flow tower
next_version = 'export NXF_VER=22.10.6'
tower = ' -with-tower '

cmd_run = f' {next_tower};{next_version};cd {execute_path}; nextflow run {nf_path} -c {param_file_path}  {tower}  -with-timeline {visualization_timeline} {resume} -w  {dir_out}   '
#-with-dag {dag_viz}
!$cmd_run